# Neural Networks

In [1]:
import numpy as np
import pandas as pd
import statistics
import random
import scipy

## A Simple example

In [2]:
class Neuron:
    def __init__(self, weights, activation_func, bias = 0):
        self.weights = weights
        self.activation = activation_func
        self.bias = bias
    def output(self, x):
        total = np.dot(self.weights, x)
        return self.activation(total + self.bias)

In [3]:
# Activation function
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [4]:
n = Neuron([0.2, 0.3, 0.4], sigmoid)
# Inputs
inp = np.array([2,1,7])
result = n.output(inp)
print(result)

0.9706877692486436


The purpose of training is to calculate (find) appropriate values for weights which is the outcome of the training.

## Layers

- Input layer: Each input is represented as a node. However, they do NOT act as a node. The input and output value is same and equal. # of nodes = # features
- Hidden layer(s): # layers and nodes are dependent on the complexity of the problem.
    - If the problem is lineary separable: No layers needed.
    - One layer is usually adequate and two layers are usually capable of solving most of the problems
    - Number of nodes: There are some guidelines available:
        1. should be less than or equal to 2*input_layer_neuron_number
        2.  
- Output layer: # of nodes depends on the problem we're trying to solve.
    - If it is a binary classification (or logistic regression) problem, single neuron is adequate. 

### How to configure?
- Only input and output layers are sufficient to solve Linearly seperable classification and/or simple regression problems.
- The definition and the number of the hidden layer are dependent on the complexity of the problem. If there is more than two hidden layers, then the network is called as **deep learning**.

In [5]:
import numpy as np
import scipy as sp
import pandas as pd
import seaborn as sns

## Diabetes Example

### Load Data

In [6]:
diabetes_data = pd.read_csv('files/diabetes.csv')

In [7]:
diabetes_data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


### Extract Data

In [8]:
x_data = diabetes_data.drop('Outcome', axis=1)
y_data = diabetes_data['Outcome']

### Split Data

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=1)

### Construct Neural Networks

1. Generate Model

In [10]:
from tensorflow.keras.models import Sequential

2024-07-08 20:27:25.544653: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
model = Sequential()

2024-07-08 20:27:26.928333: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-08 20:27:26.930086: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


2. Define Layers: **Dense** is the most frequently used layer class w/ the following API. Every neuron in the previous layer is connected to each neuron for the defined layer.
    ```python
    tensorflow.keras.layers.Dense(units, # Number of neurons
        activation=None, # Activation function to be used by all neurons in the layer. Prefer tensorflow.keras.activations if function objects to be used
        use_bias=True,
        kernel_initializer='glorot_uniform', # How to define Initial values of weights
        bias_initializer='zeros',
        kernel_regularizer=None,
        bias_regularizer=None,
        activity_regularizer=None,
        kernel_constraint=None,
        bias_constraint=None,
        **kwargs # input_dim or input_shape -> Defines neuron number for the input layer
    )
    ```
   **NOTE**: Keras does NOT allow to explicitly define input layer using layer class. Input layer is added implicitly when the first layer is added to the model by passing `input_dim` or `input_shape`.

In [12]:
from tensorflow.keras.layers import Dense

In [13]:
model.add(Dense(100, input_dim=8, activation='relu', name='Hidden_1'))
# First layer in the hidden layer with 16 neurons. This also defines the input layer with 8 neurons
# relu activation is selected. Most of the time it is sufficient

Activation functions can be passed using `tensorflow.keras.activations` module. Let's add other modules using that module. Selection of activation functions:
    - Binary Classification problems: `relu` for hidden layers and `sigmoid` for output layer

In [14]:
from tensorflow.keras.activations import relu

In [15]:
model.add(Dense(100, activation=relu, name='Hidden_2'))

In [16]:
model.add(Dense(1, activation='sigmoid', name='Output'))

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Hidden_1 (Dense)            (None, 100)               900       
                                                                 
 Hidden_2 (Dense)            (None, 100)               10100     
                                                                 
 Output (Dense)              (None, 1)                 101       
                                                                 
Total params: 11,101
Trainable params: 11,101
Non-trainable params: 0
_________________________________________________________________


Param # = # weights + # bias

3. Model compilation: 